# BAT.jl: Background + Signal Analysis 

***
In this exercise, we will get familiar with fitting a model to datasets of events, measured from your experiment, which are supplied as histograms. 

You will have access to two different datasets, which you will use separately to construct knowledge of the background of your experiment and later on to look for a signal using a Bayesian approach. The first dataset contains a large population of events and this is the one you will use to perform a Bayesian fit of the model for the background of your experiment. 

After you have obtained a model of the background, you can use the second dataset, which has a low event population, to test whether a signal is present by computing the evidence for both the background model only and the background model with a superimposed signal. 

Given the evidence for both hypotheses, you will decide whether a signal is present using the  [Bayes factor](https://en.wikipedia.org/wiki/Bayes_factor).

***


To start we need to import some packages that we will need for our analysis:

In [ ]:
using BAT
using Random
using Distributions
using IntervalSets
using ValueShapes
using Plots
using StatsBase
using LinearAlgebra
using Statistics
using ArraysOfArrays

gr(palette = :inferno, size=(750,500))

## 1. Generate Data

Let us generate `N_background=1500` samples for the background only data. And `N=440` samples for background+signal data. 

In [ ]:
bins=0:0.5:30
N_background = 1500
λ = 10

data_background = rand(Truncated(Exponential(λ), 0, 30), N_background)
hist_background = append!(Histogram(bins), data_background);

In [ ]:
N_signal = 40
N_s_background = 400
μ_true = 15.0
σ_true = 1.0

data_signal =  vcat(
    rand(Truncated(Exponential(λ), 0, 30), N_s_background),
    rand(Normal( μ_true, σ_true), N_signal)
)

hist_signal = append!(Histogram(bins), data_signal);

In [ ]:
plot(normalize(hist_background, mode=:density), alpha=0.9, color=1, fillalpha = 0.05, linewidth = 0.0, label="")
plot!(normalize(hist_background, mode=:density), seriestype = :steps, alpha=0.9, color=1,  label="Background Data")
plot!(normalize(hist_signal, mode=:density), alpha=0.9, color=3, label="Signal Data")
plot!(xlabel = "x", ylabel="counts", title="Histogram of the Data")

## 2. Definition of Model Functions & Likelihood

Given the data histograms, we have to declare the model functions we want to fit. You can notice that you can use both a scalar ($\lambda$) or a vector of variables ($A$) to define the parameters of your model. 

In [ ]:
function background_model(p::NamedTuple{(:A, :λ)}, x::Real)
    p.A[1] + p.A[2]*exp(-p.λ*x)
end

function background_signal_model(p::NamedTuple{(:A, :λ, :D, :x_0, :σ)}, x::Real)
    p.A[1] + p.A[2]*exp(-p.λ*x) + (p.D/(sqrt(2*pi*p.σ^2)))*exp(-((x - p.x_0)^2)/p.σ^2)  
end

Now, we need to define a likelihood function for our problem:

In [ ]:
function make_hist_likelihood(h::Histogram, f::Function)
    params -> begin
        # Histogram counts for each bin as an array:
        counts = h.weights

        # Histogram binning, has length (length(counts) + 1):
        binning = h.edges[1]

        # sum log-likelihood value over bins:
        ll_value::Float64 = 0.0
        
        for i in eachindex(counts)
            # Get information about current bin:
            bin_left, bin_right = binning[i], binning[i+1]
            bin_width = bin_right - bin_left
            bin_center = (bin_right + bin_left) / 2

            observed_counts = counts[i]

            # Simple mid-point rule integration of fit function `f` over bin:
            expected_counts = bin_width * f(params, bin_center)

            # Add log of Poisson probability for bin:
            ll_value += logpdf(Poisson(expected_counts), observed_counts)
            
            #!EX #=MISSING=#
            
            # Hint: Hava a look at the tutorial in the BAT.jl documentation.
        end

        return ll_value
    end 
end 

In [ ]:
algorithm = MetropolisHastings()
nsamples = 2*10^5
nchains = 5
ENV["JULIA_INFO"] = "BAT";

## 3. Background Analysis: 

Now we'll perform a Bayesian fit of the background histogram, using BAT, to infer the model parameters from the data histogram.

In [ ]:
likelihood_bm = make_hist_likelihood(hist_background, background_model);

We need to choose a sensible prior for the fit:

In [ ]:
prior_bm = NamedPrior(
    A = [0.0 .. 1000.0, 0 .. 10000.0],
    λ = 0.0 .. 20.0
);

#!EX prior_bm = NamedPrior(
#!EX     A = [#=MISSING=#, #=MISSING=#],
#!EX     λ = #=MISSING=#
#!EX );

In [ ]:
parshapes_bm = valshape(prior_bm)
posterior_bm = PosteriorDensity(likelihood_bm, prior_bm);

In [ ]:
samples_bm = bat_sample(posterior_bm, (nsamples, nchains), algorithm)
stats_bm = bat_stats(samples_bm)

In [ ]:
mode_parms_bm = parshapes_bm(stats_bm.mode)

In [ ]:
plot(samples_bm, params=[1,2,3], param_labels=["A_1","A_2","\$\\lambda\$"])

In [ ]:
plot(normalize(hist_background, mode=:density), alpha=0.9, color=1, fillalpha = 0.05, linewidth = 0.0, label="")
plot!(normalize(hist_background, mode=:density), seriestype = :steps, alpha=0.9, color=1,  label="Background Data")

plot!(xlabel = "x", ylabel="counts", title="Background Analysis")

plot!(
    0:0.1:30, x -> background_model(mode_parms_bm, x),
    label = "Background Model", lw=1.5, color=1,  linestyle = :dot
)


### Histogram as a prior: 

We would like now to use posterior distribution for $\lambda$ as a prior for signal+background fit. Let us construct histogram for $\lambda$ by using MCMC samples 

In [ ]:
samples = flatview(samples_bm.params)[3,:]
λ_hist = fit(Histogram, samples, FrequencyWeights(samples_bm.weight), nbins = 150)

## 4. Background + Signal Analysis: 

Let us use the second dataset, which has much fewer samples, to infer parameters of the signal model. Let us define a prior for the fit parameters including histogram for $\lambda$

In [ ]:
likelihood_signal = make_hist_likelihood(hist_signal, background_signal_model);

prior_signal = NamedPrior(
    A = [0.0 .. 1000.0, 0 .. 10000.0],
    λ = BAT.HistogramAsUvDistribution(λ_hist),
    D = 0.0..1000, 
    x_0 = 0 .. 30, 
    σ = 0.5 .. 10.0
);

parshapes_signal = valshape(prior_signal)
posterior_signal = PosteriorDensity(likelihood_signal, prior_signal);

In [ ]:
samples_signal = bat_sample(posterior_signal, (nsamples, nchains), algorithm)
stats_signal = bat_stats(samples_signal)

In [ ]:
plot(samples_signal, params=[1,2,3], param_labels=["A_1","A_2","\$\\lambda\$"])

In [ ]:
plot(samples_signal, params=[4,5,6], param_labels=["D","x_0","\$\\sigma\$"])

In [ ]:
plot(samples_signal, 3)
plot!(prior_signal, 3)
plot!(ylabel="P(\\lambda)", xlabel="\\lambda", title="Knowledge Update")

In [ ]:
mode_parms_signal = parshapes_signal(stats_signal.mode)

In [ ]:
plot(normalize(hist_signal, mode=:density), fillalpha=0.65, color=3, label="Signal Data")
plot!(
    0:0.1:30, x -> background_signal_model(mode_parms_signal, x),
    label = "Signal Model", lw=2, color=6, linestyle = :dash
)
plot!(xlabel = "x", ylabel="counts", title="Signal Analysis")

The Bayes factor for the signal model can be computed in the following way

In [ ]:
evidence_signal = bat_integrate(samples_signal)

## 5. Signal vs. No signal: 

Let us now analyze the same dataset under the assumption that we have no signal events

In [ ]:
likelihood_nosignal = make_hist_likelihood(hist_signal, background_model);

prior_nosignal = NamedPrior(
    A = [0.0 .. 1000.0, 0 .. 10000.0],
    λ = BAT.HistogramAsUvDistribution(λ_hist),
);

#!EX prior_nosignal = NamedPrior(
#!EX     A = [#=MISSING=#, #=MISSING=#],
#!EX     λ = #=MISSING=#
#!EX );

parshapes_nosignal = valshape(prior_nosignal)
posterior_nosignal = PosteriorDensity(likelihood_nosignal, prior_nosignal);

In [ ]:
samples_nosignal = bat_sample(posterior_nosignal, (nsamples, nchains), algorithm)
stats_nosignal = bat_stats(samples_nosignal)

In [ ]:
mode_parms_nosignal = parshapes_nosignal(stats_nosignal.mode)

In [ ]:
plot(normalize(hist_signal, mode=:density), fillalpha=0.65, color=3, label="Signal Data")
plot!(
    0:0.1:30, x -> background_model(mode_parms_nosignal, x),
    label = "Background Model", lw=2, color=1,  linestyle = :dot
)
plot!(
    0:0.1:30, x -> background_signal_model(mode_parms_signal, x),
    label = "Signal Model", lw=2, color=6, linestyle = :dash
)
plot!(xlabel = "x", ylabel="counts", title="Signal Analysis")

The Bayes factor for this case is equal to

In [ ]:
evidence_nosignal = bat_integrate(samples_nosignal)

Finally, let us compare Bayes factors for these two assumptions: 

In [ ]:
evidence_signal / evidence_nosignal

    Tasks: 
    
    1) Change the number of signal/background events and see for which values we no longer detect the signal. Explore how the binning of the data effects analysis. 
    
    2) Assume that we do not have the dataset with the background only measurements, and we use a flat prior for the parameter lambda. Explore how this effects MCMC sampling. 